# Data Collection - PART 2
## Notebook to scrap each page 
We grabed links in bulk form the banners on overstock in the previous notebook. In this  notebook we cycle through each link visit he page and get product description along with product images for each product.

Note: I made page_scrap_category.ipynb mulitple such notebooks so that I can parallely scrap different product categories at the same time. 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import math
import os
chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

### 1. OPEN LIST OF LINKS

In [2]:
prod_list=pd.read_csv('../data/scrape/product_links_sofa.csv')
prod_list.drop('Unnamed: 0',axis =1, inplace = True)
print(prod_list.shape)
prod_list.tail()                    

(2840, 2)


,link,type
2835,https://www.overstock.com/Home-Garden/SofaTren...,sofa
2836,https://ad.tagdelivery.com/click?id=b2b66ebb-2...,sofa
2837,https://ad.tagdelivery.com/click?id=d9e56781-8...,sofa
2838,https://ad.tagdelivery.com/click?id=74a0ee6f-f...,sofa
2839,https://ad.tagdelivery.com/click?id=66d3d3d7-d...,sofa


### 2. SETTING UP 
Setting up all the headings/parameters I need two empty dataframes . 1. to store product details. 2. to store the image links

In [19]:
# list out all heading, parameters we will get from scraping a page.  
heading1 =['page_link','product_name','item_no','description','style',\
          'material','type_class','color','rating']

#initialize an empty dataframe
df_product=pd.DataFrame(columns=heading1)
df_product

,page_link,product_name,item_no,description,style,material,type_class,color,rating


In [20]:
# list out all heading, parameters we will get from scraping a page. 
# add product_name and page_link so that later on i can join it with the product details  
heading2 =['image_link','product_name','page_link']

#initialize an empty dataframe
df_images=pd.DataFrame(columns=heading2)
df_images

,image_link,product_name,page_link


### 2. Scrap it

In [21]:
#counter to check where we are, track if we ge interrupted
counter=1001

In [22]:
# condition for x_path

driver = webdriver.Chrome(chromedriver)
#loop through the product_list we have made.  manually set start and finish. Use exit counter to know where you are at 
for link_str in prod_list.loc[1001:,'link']:
    
    print('----START PAGE ----')
    print(link_str)
    print('COUNT= ',counter)
    driver.get(str(link_str))
    time.sleep(2)
        
    page_link = link_str
    
    # get product name
    product_name_selector = '//h1'
    product_name = driver.find_element_by_xpath(product_name_selector).text
    #print(product_name)
    
    # get item_no
    item_no_selector ='//p[@class ="description-item-number"]'
    item_no=driver.find_element_by_xpath(item_no_selector).text.split()[1]
    item_no = int(item_no)
    #print(item_no)
    
    # get description
    description_selector ='//span[@itemprop ="description"]'
    description=driver.find_element_by_xpath(description_selector).text.split('\n')[0]
    #print(description)
    
    # get style , material, type_class, color
    style_selector = '//td[contains(text(),"Style")]/following-sibling::td'
    material_selector = '//td[contains(text(),"Material")]/following-sibling::td'
    type_class_selector = '//td[contains(text(),"Type")]/following-sibling::td'
    color_selector = '//td[contains(text(),"Color")]/following-sibling::td'
    
    try:
        style = driver.find_element_by_xpath(style_selector).text.strip()
    except:
        style = ""       
    try:
        material = driver.find_element_by_xpath(material_selector).text.strip()
    except:
        material = ""    
    try:
        type_class = driver.find_element_by_xpath(type_class_selector).text.strip()
    except:
        type_class = ""
    try:
        color = driver.find_element_by_xpath(color_selector).text.strip()
    except:
        color = ""


    
    # get rating
    rating_selector ='//span[@class ="overall-rating"]'
    try:
        rating=driver.find_element_by_xpath(rating_selector).text
    except:
        rating = 0
    
    rating = float(rating)


    temp_list = [page_link,product_name,item_no,description,style,material,type_class,color,rating]
    try:
        # put into a  first panda
        df_product = df_product.append(pd.DataFrame([temp_list], columns=heading1),ignore_index=True)
    except:
        print('OPPS A ... no put in pandas')
    

    ###### images #####
    image_panel = '//div[@class="gallery-controls"]//div[@class ="thumb-frame"]//child::li'
    for row in driver.find_elements_by_xpath(image_panel):
        image_link = row.get_attribute('data-max-img')
        temp_img_list = [image_link,product_name,page_link]
        try:
        # put into a second panda
            df_images = df_images.append(pd.DataFrame([temp_img_list], columns=heading2),ignore_index=True)
        except:
            print('OPPS B ... no put in pandas')      
    
    counter +=1
    print('----END PAGE-----')
    
driver.close()
  

----START PAGE ----
https://www.overstock.com/Home-Garden/Emerald-Home-Navaro-Grey-Motion-Sofa/18045867/product.html?refccid=LZNGZNFTGF5L5ZKNYNQJN3LHZA&searchidx=1001
COUNT=  1001
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Abbyson-Ashton-Top-Grain-Leather-Brown-Sofa/10069431/product.html?refccid=LZNGZNFTGF5L5ZKNYNQJN3LHZA&searchidx=1002
COUNT=  1002
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Kardiel-Basil-Modern-Loft-Sofa/12808511/product.html?refccid=LZNGZNFTGF5L5ZKNYNQJN3LHZA&searchidx=1003
COUNT=  1003
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Handy-Living-Orlando-SoFast-Medium-Blue-Hereringbone-Slipcover-Sofa/21723985/product.html?refccid=LZNGZNFTGF5L5ZKNYNQJN3LHZA&searchidx=1004
COUNT=  1004
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Mid-Century-Modern-Upholstered-Tufted-Back-Single-Cushion-Sofa/23136413/product.html?refccid=LZNGZNFTGF5L5ZKNYNQJN3LHZ

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Luca-Home-Ivory-Crystal-Embellished-Tufted-Sofa/10136127/product.html?refccid=FKOSOVLFMBEYUZSLNGIK3HGI7U&searchidx=1040
COUNT=  1040
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Abbyson-Beliz-Brown-Leather-Sofa/9963794/product.html?refccid=FKOSOVLFMBEYUZSLNGIK3HGI7U&searchidx=1041
COUNT=  1041
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/JAR-Designs-The-Huntley-Taupe-Truffle-Sofa/4803956/product.html?refccid=FKOSOVLFMBEYUZSLNGIK3HGI7U&searchidx=1042
COUNT=  1042
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Simmons-Upholstery-Albany-Slate-Sofa-Chaise/22635169/product.html?refccid=FKOSOVLFMBEYUZSLNGIK3HGI7U&searchidx=1043
COUNT=  1043
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Lafontaine-Sofa-Chambray/12653654/product.html?refccid=FKOSOVLFMBEYUZSLNGIK3HGI7U&searchidx=1044
COUNT=  1044
----

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Jacobs-Sofa/18025116/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1080
COUNT=  1080
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Torino-Sofa-Slate-Walnut/23131692/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1081
COUNT=  1081
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Serta-Andrea-Convertible-Sofa-by-Lifestyle-Solutions/18003221/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1082
COUNT=  1082
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Rocky-Sofa-White/18526616/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1083
COUNT=  1083
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Handy-Living-Millona-SoFast-Dark-Blue-Microfiber-Compact-Sofa/24312779/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1084
COUNT=  1084
----END PAGE-----
----STAR

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Handy-Living-Bella-SoFast-Heather-Grey-Herringbone-Slipcover-Sofa/21725174/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1119
COUNT=  1119
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Porter-Casper-Blue-Mid-Century-Modern-Tufted-Sofa/15274293/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1120
COUNT=  1120
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/ProLounger-Dark-Grey-Tufted-Velvet-3-Seat-Recliner-Sofa/23463782/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1121
COUNT=  1121
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Aurelle-Home-Velvet-Blue-Meredith-Chaise-Sofa/11370745/product.html?refccid=3CBA7HSFBAPYHFSN5HI22OS77I&searchidx=1122
COUNT=  1122
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Simon-Li-Hemer-Top-Grain-Leather-Sofa/23566133/product.html?re

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Southern-Motion-Major-League-Taupe-Leather-Double-Reclining-Sofa-with-Power-Headrest/22655946/product.html?refccid=K4W57OFBH33KRO2DBOZ7GYULEU&searchidx=1159
COUNT=  1159
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Nancy-Slipcover-Eco-Friendly-Sofa/14587677/product.html?refccid=K4W57OFBH33KRO2DBOZ7GYULEU&searchidx=1160
COUNT=  1160
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Townsend-Sofa-by-Greyson-Living/11142320/product.html?refccid=K4W57OFBH33KRO2DBOZ7GYULEU&searchidx=1161
COUNT=  1161
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Panama-Jack-Sanibel-Sofa/9962216/product.html?refccid=K4W57OFBH33KRO2DBOZ7GYULEU&searchidx=1162
COUNT=  1162
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Chic-Home-Parker-Button-Tufted-Velvet-Wood-Frame-Sofa/22884411/product.html?refccid=K4W57OFBH33KRO2DBOZ7G

...

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Reclining-Loveseat-Charcoal-Grey-Bonded-Leather/10898182/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2653
COUNT=  2661
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Emerald-Home-Clarkson-Espresso-Sofa-with-2-Pillows/18045844/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2654
COUNT=  2662
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Nicolai-Traditional-Tan-Espresso-Upholstered-Sofa/19576450/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2655
COUNT=  2663
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Furniture-of-America-Cara-Contemporary-Ivory-Tufted-Sofa/10353154/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2656
COUNT=  2664
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Lorenzo-Sofa/20876810/product.html?refccid=IVNJHQW3YG

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Velvet-Fabric-Cushiony-Adjustable-Sofa-In-Gray/20649381/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2691
COUNT=  2699
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Standard-Furniture-Sundance-Almond-Faux-Leather-Poly-PU-Blend-Manual-Motion-Sofa/15858993/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2692
COUNT=  2700
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Skyler-Transitional-Style-Comfy-Sofa-Ivory/18231606/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2693
COUNT=  2701
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/J-M-Lorenzo-Caramel-Leather-Sofa/21174321/product.html?refccid=IVNJHQW3YG53JCSAXXXKSMZLEU&searchidx=2694
COUNT=  2702
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/reclining-sofa-black-bonded-leather/10878719/product.html?refccid=IVNJHQW3YG

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Black-Bonded-Leather-Reclining-Sofa/10898120/product.html?refccid=3AUZ2NBVAEMIYTSC7MHCH5ECNU&searchidx=2719
COUNT=  2731
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/King-Sofa/20876728/product.html?refccid=3AUZ2NBVAEMIYTSC7MHCH5ECNU&searchidx=2720
COUNT=  2732
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Emerald-Marquis-Walnut-Sofa/12754039/product.html?refccid=3AUZ2NBVAEMIYTSC7MHCH5ECNU&searchidx=2721
COUNT=  2733
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Parma-Modern-Plush-Cushion-Sofa-Black/18231789/product.html?refccid=3AUZ2NBVAEMIYTSC7MHCH5ECNU&searchidx=2722
COUNT=  2734
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Flannel-Fabric-Adjustable-Sofa-Green/18854603/product.html?refccid=3AUZ2NBVAEMIYTSC7MHCH5ECNU&searchidx=2723
COUNT=  2735
----END PAGE-----
----START PAGE ----
https://

----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=d9e56781-85c4-490b-913e-d253f4816664&x=5qWi59pnsnfEAOORgYNJb76LWgJXd-dO7aht947tq6YFP9hv2kJ6zBcMHtsmpTiz4Sdfky7vVS68cQxoxizTajkThOiXHItQBrVaUnr8jkzn7XVRRRK20bKQPt_-HaTHteLiTcvQzGN-hVhBJagUgKk9AqpZe2uSJw6GW3T3wQFhLGkSyIOYtG-GIfl0HOvgqNFWlRsYeqfHf0j5Dv3L-oPnR5EvFP78LwerixjU7wqRh4O3iKqJ6tPcz_FMNrs5aprISlQugdkBOAWvxVFdCw%3D%3D&redirect=https%3A%2F%2Fwww.overstock.com%2FHome-Garden%2FPorch-Den-Echo-Park-Quintero-Tufted-Velvet-Sofa%2F14255823%2Fproduct.html%3Frefccid%3DPX4PFQAJDYSZHICIPDNE4BRIQI%26searchidx%3D1
COUNT=  2773
----END PAGE-----
----START PAGE ----
https://ad.tagdelivery.com/click?id=74a0ee6f-ff95-4227-9f6c-e825c6179639&x=zZK3r_MEzwVhUh_BoznIbgq9ogknMhVcTxu7amjWViT9Q-16wQdP0wVrK4gfZWtcjKs1Z_PistUKNLVftEZX30xq8BEVPV8ipZnTUw4qz3gE_tMq3TxHhbkmIVNjM3f-GT4-cFuut9eEci9gup_jb5KqIY_7RKWJ8lhQOE4wgDwj2vehaU6M5yLNbxMnLjMSjSbRydn93MT-CAiTSI76WYudhz6lAKiAeu36C6qklJnyzh9qHJ-_BL26gxNF5r5wd1jxUw2mXHBl3iMWn6B0ew%3D%3D&redir

----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Arima-Power-Sofa-Recliner-With-USB-Charging-Port/25627704/product.html?refccid=ZJ6YDFFUZBC2UB2IC3VTOWSA3I&searchidx=2791
COUNT=  2807
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Mercana-Armelle-III-Leather-Sofa/25628377/product.html?refccid=ZJ6YDFFUZBC2UB2IC3VTOWSA3I&searchidx=2792
COUNT=  2808
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Mercana-Chauncey-II-Leather-Sofa/25628378/product.html?refccid=ZJ6YDFFUZBC2UB2IC3VTOWSA3I&searchidx=2793
COUNT=  2809
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Torrey-Sofa/25573085/product.html?refccid=ZJ6YDFFUZBC2UB2IC3VTOWSA3I&searchidx=2794
COUNT=  2810
----END PAGE-----
----START PAGE ----
https://www.overstock.com/Home-Garden/Easy-Living-Swiss-Dual-Reclining-Sofa-with-USB/25483283/product.html?refccid=ZJ6YDFFUZBC2UB2IC3VTOWSA3I&searchidx=2795
COUNT=  2811
----END PAGE-----
---

In [23]:
# what have I got ?
print(counter)
print(df_product.shape)
df_product.tail()

2840
(1839, 9)


,page_link,product_name,item_no,description,style,material,type_class,color,rating
1834,https://www.overstock.com/Home-Garden/SofaTren...,SofaTrendz Frisco Denim Blue Sofa,30874963,Invite friends and family to sit and visit for...,Modern & Contemporary,Polyester,Sofa,Blue,0.0
1835,https://ad.tagdelivery.com/click?id=b2b66ebb-2...,Clay Alder Home Harrison Mid-century Modern Sofa,18896060,Give your home a sophisticated flair with this...,"Americana, Contemporary, Mid-Century, Modern","Fabric, Foam, Polyester",Sofa,"Blue, Grey",4.2
1836,https://ad.tagdelivery.com/click?id=d9e56781-8...,Porch & Den Echo Park Quintero Tufted Velvet Sofa,20844033,Bring luxury and charm into your seating area ...,Recessed Arms,"Foam, Polyester, Rubberwood",Sofa,"Blue, Green, Grey, Purple, Red",4.5
1837,https://ad.tagdelivery.com/click?id=74a0ee6f-f...,Seatcraft Republic Leather Home Theater Seatin...,21887056,The Seatcraft Republic home theater seat boast...,Modern & Contemporary,Leather,Sofa,Brown,0.0
1838,https://ad.tagdelivery.com/click?id=66d3d3d7-d...,Porch & Den Echo Park Quintero Vinyl Sofa,20844032,Traditional elegance meets mid-century modern ...,Mid-Century,"Foam, Plastic, Polyester, Rubberwood, Vinyl, Wood",Sofa,"Blue, Grey",4.0


In [24]:
df_product['page_link'].nunique()

1823

In [25]:
print(df_images['page_link'].nunique())

1823


In [26]:
df_images.tail()

,image_link,product_name,page_link
8115,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
8116,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
8117,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
8118,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
8119,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...


### 3. WRITE DATA TO FILE
If the file does not exist then i write the dataframe with headers else if the file is already there then don't append with the header row 

In [27]:
file_name1 = '../data/scrape/product_details_sofa.csv'
from pathlib import Path
config = Path(file_name1)
if config.is_file():
    print('exist = yes')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name1, 'a') as f:
        df_product.to_csv(f, header=True)
    

exist = yes


In [28]:
file_name2 = '../data/scrape/image_data_sofa.csv'
from pathlib import Path
config = Path(file_name2)
if config.is_file():
    print('exist = yes')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=False)
    # Store configuration file values
else:
    # Keep presets
    print('exists = no')
    with open(file_name2, 'a') as f:
        df_images.to_csv(f, header=True)
    

exist = yes


### 4. CHECK THE APPENDED FILE 
Read the file and check if stuff was written 

In [29]:
df_check1 = pd.read_csv(file_name1)
print(df_check1.shape)
df_check1.tail()

(2839, 10)


,Unnamed: 0,page_link,product_name,item_no,description,style,material,type_class,color,rating
2834,1834,https://www.overstock.com/Home-Garden/SofaTren...,SofaTrendz Frisco Denim Blue Sofa,30874963,Invite friends and family to sit and visit for...,Modern & Contemporary,Polyester,Sofa,Blue,0.0
2835,1835,https://ad.tagdelivery.com/click?id=b2b66ebb-2...,Clay Alder Home Harrison Mid-century Modern Sofa,18896060,Give your home a sophisticated flair with this...,"Americana, Contemporary, Mid-Century, Modern","Fabric, Foam, Polyester",Sofa,"Blue, Grey",4.2
2836,1836,https://ad.tagdelivery.com/click?id=d9e56781-8...,Porch & Den Echo Park Quintero Tufted Velvet Sofa,20844033,Bring luxury and charm into your seating area ...,Recessed Arms,"Foam, Polyester, Rubberwood",Sofa,"Blue, Green, Grey, Purple, Red",4.5
2837,1837,https://ad.tagdelivery.com/click?id=74a0ee6f-f...,Seatcraft Republic Leather Home Theater Seatin...,21887056,The Seatcraft Republic home theater seat boast...,Modern & Contemporary,Leather,Sofa,Brown,0.0
2838,1838,https://ad.tagdelivery.com/click?id=66d3d3d7-d...,Porch & Den Echo Park Quintero Vinyl Sofa,20844032,Traditional elegance meets mid-century modern ...,Mid-Century,"Foam, Plastic, Polyester, Rubberwood, Vinyl, Wood",Sofa,"Blue, Grey",4.0


In [30]:
df_check2 = pd.read_csv(file_name2)
print(df_check2.shape)
print(df_check2['page_link'].nunique())
df_check2.tail()

(16333, 4)
2823


,Unnamed: 0,image_link,product_name,page_link
16328,8115,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
16329,8116,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
16330,8117,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
16331,8118,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...
16332,8119,https://ak1.ostkcdn.com/images/products/142558...,Porch & Den Echo Park Quintero Vinyl Sofa,https://ad.tagdelivery.com/click?id=66d3d3d7-d...


### next -> ../code/data_clean_join.ipynb 